# GNSS–Moon Ray Plasmaspheric Integration (PyGCPM)

---

## 1 Environment setup (Colab / Linux)

Mount Drive, install dependencies, clone and compile PyGCPM, copy required data files.

In [ ]:
# ============================================================
# Environment setup (Colab / Linux)
# ============================================================

from google.colab import drive
drive.mount("/content/drive")

# ------------------------------------------------------------
# Python dependencies
# ------------------------------------------------------------
!pip install -q pytecgg spacepy jplephem astropy cartopy polars tqdm

# ------------------------------------------------------------
# PyGCPM installation and compilation, and src code
# ------------------------------------------------------------

! git clone https://github.com/viventriglia/iono-lugre.git

# Manual installation of the plasmaspheric model and executable compilation (this is much easier on Linux)
! git clone https://github.com/mattkjames7/PyGCPM
! cp /content/drive/Shareddrives/Iono_LuGRE/GPCM/data/*.dat PyGCPM/PyGCPM/__data/libgcpm/iri/
! pip install -q -e PyGCPM
! cp /content/drive/Shareddrives/Iono_LuGRE/GPCM/data/Kp_ap_since_1932.txt .
%cd /content/PyGCPM/PyGCPM/__data/libgcpm/
! make clean -i > /dev/null
! make > /dev/null
%cd /content/


## 2 Imports, paths, and project modules

Import scientific libraries and project modules.  
Define data, cache, plot, and Kp file paths.


In [ ]:
# ============================================================
# Imports and configuration
# ============================================================

from pathlib import Path
from datetime import datetime
import sys

import numpy as np
import pandas as pd
import polars as pl
from tqdm.notebook import tqdm

# GNSS / ephemerides
from pytecgg.satellites import prepare_ephemeris, satellite_coordinates
from pytecgg.parsing import read_rinex_nav

from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, get_body, ITRS

from PyGCPM import PyGCPM
from spacepy import coordinates as coord
from spacepy.time import Ticktock

solar_system_ephemeris.set("jpl")

PROJECT_SRC = Path("/content/iono-lugre/src")
sys.path.insert(0, str(PROJECT_SRC))

from processing_func import run_segment_analysis
from plotting_func import plot_integrated_results
from utils_func import parse_arc_id, kp_extraction_function, calculate_tangent_points


# ------------------------------------------------------------
# Paths
# ------------------------------------------------------------

DATA_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/GPCM/data/")
CACHE_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/GPCM/cache_runs/")
PLT_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC/GCPM_Plots")
PLT_UTILS_PATH = Path("/content/drive/Shareddrives/Iono_LuGRE/Plots_GFLC")

KP_FILE = Path("/content/Kp_ap_since_1932.txt")


## 3 Arc selection and time grid

Select the GNSS arc of interest, parse satellite/constellation, load observations,  
define the uniform time grid, and list required navigation files.

In [ ]:
# ============================================================
# Arc selection
# ============================================================

arc_of_interest = "G30_OP74_102"
sv_id, constellation = parse_arc_id(arc_of_interest)

# Load observation table
obs_tab = pd.read_csv(PLT_UTILS_PATH / "obs_tab_new.csv")

sel = obs_tab.loc[obs_tab["ArcID"] == arc_of_interest].copy()
sel["time"] = pd.to_datetime(sel["time"], errors="raise")

unique_doy = sel["time"].dt.dayofyear.unique()

# Uniform time grid (30 s)
times = pl.Series(
    pd.date_range(
        start=sel["time"].min(),
        end=sel["time"].max(),
        freq="30s"
    ).to_list()
)

# Navigation files
nav_files = [
    DATA_PATH / f"BRDC00IGS_R_2025{doy:03d}0000_01D_MN.rnx"
    for doy in unique_doy
]


## 4 Satellite coordinates from broadcast ephemeris

Compute satellite Cartesian coordinates along the arc using RINEX broadcast ephemerides.

In [ ]:
# ============================================================
# Satellite coordinates from broadcast ephemeris
# ============================================================

sat_x_all, sat_y_all, sat_z_all = [], [], []

for nav_file in nav_files:
    nav_dict = read_rinex_nav(str(nav_file))
    ephem_dict = prepare_ephemeris(nav_dict, constellation=constellation)

    sat_coords = satellite_coordinates(
        ephem_dict=ephem_dict,
        sv_ids=pl.Series([sv_id] * len(times)),
        gnss_system=constellation,
        epochs=times,
    )

    sat_x_all.append(np.asarray(sat_coords["sat_x"]))
    sat_y_all.append(np.asarray(sat_coords["sat_y"]))
    sat_z_all.append(np.asarray(sat_coords["sat_z"]))

sat_x = np.concatenate(sat_x_all)
sat_y = np.concatenate(sat_y_all)
sat_z = np.concatenate(sat_z_all)


## 5 Moon position in Earth-fixed frame (ITRS)

Compute Moon Cartesian coordinates in the Earth-fixed (ITRS) frame for the same epochs.


In [ ]:
# ============================================================
# Moon position in ECEF (ITRS)
# ============================================================

times_astropy = Time(times.to_list(), scale="utc")

moon_icrs = get_body("moon", times_astropy)
moon_ecef = moon_icrs.transform_to(ITRS(obstime=times_astropy))


## 6 Ray construction (satellite → Moon)

Build ray segments connecting satellite positions to Moon positions in Cartesian km.

In [ ]:
# ============================================================
# Build ray segments (satellite → Moon)
# ============================================================

list_of_segments = []

for i in range(len(moon_ecef)):
    P1 = [
        sat_x[i] / 1e3,
        sat_y[i] / 1e3,
        sat_z[i] / 1e3,
    ]  # km

    P2 = [
        moon_ecef.x.value[i],
        moon_ecef.y.value[i],
        moon_ecef.z.value[i],
    ]  # km

    list_of_segments.append((P1, P2))


## 7 Segment mass analysis (cached)

Run or load cached plasmaspheric mass integration through the spherical shell.

In [ ]:
# ============================================================
# Segment mass analysis (cached)
# ============================================================

R_Earth = 6371.0
R1_th = R_Earth
R2_th = R_Earth + 40000
N_steps_mass = 100

CACHE_PATH.mkdir(parents=True, exist_ok=True)
result_path = CACHE_PATH / f"results_{arc_of_interest}.npy"

if result_path.exists():
    results = np.load(result_path, allow_pickle=True)
else:
    results = run_segment_analysis(
        segments=list_of_segments,
        times=times.to_list(),
        N_steps_mass=N_steps_mass,
        R1=R1_th,
        R2=R2_th,
        R_Earth_km=R_Earth,
        kp_extraction_function=lambda t: kp_extraction_function(t, KP_FILE),
        PyGCPM=PyGCPM,
        Coords=coord.Coords,
        Ticktock=Ticktock,
        tqdm=tqdm,
    )
    np.save(result_path, results)

## 8 Tangent point computation and metadata preparation

Compute tangent points and heights and prepare LT/UT metadata for plotting.

In [ ]:
# ============================================================
# Tangent point computation
# ============================================================


tp_coords, tp_heights = calculate_tangent_points(
    sel,
    R_Earth_km=R_Earth,
)

sel["tp_x"] = tp_coords[:, 0]
sel["tp_y"] = tp_coords[:, 1]
sel["tp_z"] = tp_coords[:, 2]
sel["tp_h"] = tp_heights

# ------------------------------------------------------------
# Time formatting for plotting
# ------------------------------------------------------------

sel["lt_ip"] = sel["lt_ip"].str.slice(0, 5)
sel["ut_time"] = sel["time"].dt.strftime("%H:%M")


## 9 Integrated results visualization

Visualize integrated results using the plotting utilities.

In [ ]:
plot_integrated_results(
    results=results,
    times=times.to_list(),
    sel=sel,
    title=arc_of_interest,
    show_colorbar=True,
    left_letter="g",
    right_letter="h",
    save=False,
    font_scale=1.5,
    n_geodetics=20,
    label_every=3,
    legend_loc="center left",
)
